In [1]:
# using NLopt
using Optim
using StatsBase
using Combinatorics

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl");

Ok, so I think the way this works is that `myfunc` is going to supply the update in the gradient by modifying it in place. This seems very smart -- each function call updates the gradient. So, what we would need to do evaluate the relevant sums and cut terms, then follow this pattern. 

In [58]:
# parameters

n = 100
Z = rand(1:3, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = maximum(1.0*p).^α[1] / (sum((1.0*p).^α[1])*α[2])

α0 = [1.0, 10000]

kmax = 3

Ω = buildΩ(ω, α0, kmax)

Ω (generic function with 1 method)

In [59]:
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((100,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
  E: Dict{Int64,Dict}
  D: Array{Int64}((100,)) [6, 13, 10, 3, 2, 5, 4, 7, 5, 4  …  5, 5, 8, 5, 4, 2, 2, 3, 8, 3]


In [72]:
function obj(x)
    """
    objective for optimization. Look into gradients later. 
    """
    Q = modularity(H::hypergraph, Z::Array{Int64,1}, Ω; α=x, bigInt=true)
#     println(Float64(Q))
    return(Float64(-Q))
end

obj (generic function with 2 methods)

In [92]:
res = optimize(obj, 0, [10.0, 1000.0],
    NelderMead(),
    Optim.Options(f_tol = 1e-10,
        iterations = 1000,
        show_trace=false))
α̂ = Optim.minimizer(res)

2-element Array{Float64,1}:
      0.9431769219857955
 670079.1365972597

In [93]:
# true
logLikelihood(H, Z, Ω, ϑ; α=α0) 

-1539.5602703062598

In [94]:
# before estimating α
logLikelihood(H, Z, Ω, 1.0*H.D; α=α0)

-13973.912319160463

In [95]:
# after estimating α
logLikelihood(H, Z, Ω, 1.0*H.D; α=α̂)

-1489.4745254409484